In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

Collect Data + Preprocess Data

In [2]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0][['Symbol', 'GICS Sector']]
tickers['Symbol'] = tickers['Symbol'].str.replace('.', '-')
sector_breakdown = tickers.groupby('GICS Sector')['Symbol'].apply(list)
sector_breakdown = sector_breakdown.to_dict()

# Print the sector breakdown
print(sector_breakdown)

{'Communication Services': ['GOOGL', 'GOOG', 'T', 'CHTR', 'CMCSA', 'DIS', 'EA', 'FOXA', 'FOX', 'IPG', 'LYV', 'MTCH', 'META', 'NFLX', 'NWSA', 'NWS', 'OMC', 'PARA', 'TMUS', 'TTWO', 'VZ', 'WBD'], 'Consumer Discretionary': ['ABNB', 'AMZN', 'APTV', 'AZO', 'BBWI', 'BBY', 'BKNG', 'BWA', 'CZR', 'KMX', 'CCL', 'CMG', 'DHI', 'DRI', 'DPZ', 'EBAY', 'ETSY', 'EXPE', 'F', 'GRMN', 'GM', 'GPC', 'HAS', 'HLT', 'HD', 'LVS', 'LEN', 'LKQ', 'LOW', 'LULU', 'MAR', 'MCD', 'MGM', 'MHK', 'NKE', 'NCLH', 'NVR', 'ORLY', 'POOL', 'PHM', 'RL', 'ROST', 'RCL', 'SBUX', 'TPR', 'TSLA', 'TJX', 'TSCO', 'ULTA', 'VFC', 'WHR', 'WYNN', 'YUM'], 'Consumer Staples': ['ADM', 'MO', 'BF-B', 'BG', 'CPB', 'CHD', 'CLX', 'KO', 'CL', 'CAG', 'STZ', 'COST', 'DG', 'DLTR', 'EL', 'GIS', 'HSY', 'HRL', 'K', 'KVUE', 'KDP', 'KMB', 'KHC', 'KR', 'LW', 'MKC', 'TAP', 'MDLZ', 'MNST', 'PEP', 'PM', 'PG', 'SJM', 'SYY', 'TGT', 'TSN', 'WBA', 'WMT'], 'Energy': ['APA', 'BKR', 'CVX', 'COP', 'CTRA', 'DVN', 'FANG', 'EOG', 'EQT', 'XOM', 'HAL', 'HES', 'KMI', 'MRO', '

In [3]:
ticker_list = []
for sector in sector_breakdown:
    ticker_list.extend(sector_breakdown[sector])

today = pd.Timestamp.today().strftime('%Y-%m-%d')
month_ago = pd.Timestamp.today() - pd.DateOffset(months=1)

data = yf.download(ticker_list, start=month_ago, end=today)
#data = data.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'])
data

[********              16%%                      ]  79 of 503 completed

In [ ]:
# Backward fill NaN values
data = data.fillna(method='bfill', axis=0)

# Backward fill NaN values
data = data.fillna(method='ffill', axis=0)

# Print number of NA values across each column
print(data.isna().sum(axis=1))

C:\Users\ayush\AppData\Local\Temp\ipykernel_27672\1922356623.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='bfill', axis=0)
C:\Users\ayush\AppData\Local\Temp\ipykernel_27672\1922356623.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill', axis=0)


Date
2023-10-30 00:00:00    54
2023-10-31 00:00:00    54
2023-11-01 00:00:00    54
2023-11-02 00:00:00    54
2023-11-03 00:00:00    54
2023-11-06 00:00:00    54
2023-11-07 00:00:00    54
2023-11-08 00:00:00    54
2023-11-09 00:00:00    54
2023-11-10 00:00:00    54
2023-11-13 00:00:00    54
2023-11-14 00:00:00    54
2023-11-15 00:00:00    54
2023-11-16 00:00:00    54
2023-11-17 00:00:00    54
2023-11-20 00:00:00    54
2023-11-21 00:00:00    54
2023-11-22 00:00:00    54
2023-11-24 00:00:00    54
2023-11-27 00:00:00    54
2023-11-28 00:00:00    54
2023-11-29 00:00:00    54
dtype: int64


In [ ]:
# Calculate daily returns
returns = data.pct_change()

# Drop the index name Adj Close
returns = returns.droplevel(0, axis=1)

# Group tickers in returns dataframe by sector using sector_breakdown's keys & values
sector_mean_returns = {}

# Extract each key & value pair from sector_breakdown
for sector, tickers in sector_breakdown.items():
    # Get tickers for each sector
    sector_data = returns[tickers]

    # Calculate mean returns for each sector and add to dictionary
    sector_mean_returns[sector] = sector_data.mean(axis=1)

# Convert sector_mean_returns dict to a DataFrame
sector_mean_returns = pd.DataFrame(sector_mean_returns)

sector_mean_returns

C:\Users\ayush\AppData\Local\Temp\ipykernel_27672\677370838.py:2: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  returns = data.pct_change()


,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
Date,,,,,,,,,,,
2023-10-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-10-31 00:00:00,0.006818,0.010581,0.051700,0.000853,0.033825,0.023130,0.054567,0.017828,0.036551,0.028251,0.046674
2023-11-01 00:00:00,0.023748,0.037689,0.012018,0.026973,0.015783,0.003006,0.028862,0.044617,0.023466,-0.014274,0.042803
2023-11-02 00:00:00,0.038996,0.040219,0.014530,0.033533,0.053960,0.028057,0.024113,0.035161,0.001615,0.048559,0.024123
2023-11-03 00:00:00,0.017264,0.018826,-0.003300,-0.001097,0.010217,0.017613,0.000788,0.008732,0.010025,0.026188,0.011034
2023-11-06 00:00:00,-0.030658,-0.026994,-0.005358,-0.035832,-0.033771,-0.006510,-0.031583,-0.023917,-0.010962,-0.038976,-0.012293
2023-11-07 00:00:00,0.003967,0.005343,-0.016675,0.054382,-0.015512,0.010882,0.008701,0.013361,0.053620,-0.000681,-0.011935
2023-11-08 00:00:00,0.070898,0.005408,0.037301,-0.017635,0.002474,0.006304,0.017837,0.000945,-0.017990,-0.011569,0.022827
2023-11-09 00:00:00,-0.002383,-0.001373,-0.006212,-0.030868,0.012759,0.000774,0.011196,0.017190,0.021756,0.043260,-0.004376


In [ ]:
# Drop rows that have NaN values
sector_mean_returns = sector_mean_returns.dropna()
sector_mean_returns

,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
Date,,,,,,,,,,,
2023-10-31 00:00:00,0.006818,0.010581,0.051700,0.000853,0.033825,0.023130,0.054567,0.017828,0.036551,0.028251,0.046674
2023-11-01 00:00:00,0.023748,0.037689,0.012018,0.026973,0.015783,0.003006,0.028862,0.044617,0.023466,-0.014274,0.042803
2023-11-02 00:00:00,0.038996,0.040219,0.014530,0.033533,0.053960,0.028057,0.024113,0.035161,0.001615,0.048559,0.024123
2023-11-03 00:00:00,0.017264,0.018826,-0.003300,-0.001097,0.010217,0.017613,0.000788,0.008732,0.010025,0.026188,0.011034
2023-11-06 00:00:00,-0.030658,-0.026994,-0.005358,-0.035832,-0.033771,-0.006510,-0.031583,-0.023917,-0.010962,-0.038976,-0.012293
2023-11-07 00:00:00,0.003967,0.005343,-0.016675,0.054382,-0.015512,0.010882,0.008701,0.013361,0.053620,-0.000681,-0.011935
2023-11-08 00:00:00,0.070898,0.005408,0.037301,-0.017635,0.002474,0.006304,0.017837,0.000945,-0.017990,-0.011569,0.022827
2023-11-09 00:00:00,-0.002383,-0.001373,-0.006212,-0.030868,0.012759,0.000774,0.011196,0.017190,0.021756,0.043260,-0.004376
2023-11-10 00:00:00,0.004671,0.002440,0.011980,0.010144,-0.001671,0.020273,0.001336,0.019270,-0.008436,-0.000985,-0.023267


In [ ]:
# Calculate the variance-covariance matrix for the 11 sector returns
cov_matrix = sector_mean_returns.cov()
cov_matrix

,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
Communication Services,0.004709,0.004117,0.003711,0.003240,0.004203,0.003841,0.003768,0.003587,0.005120,0.005444,0.004754
Consumer Discretionary,0.004117,0.004390,0.003463,0.003282,0.004403,0.003930,0.003802,0.003671,0.004975,0.005627,0.004560
Consumer Staples,0.003711,0.003463,0.003375,0.002668,0.003557,0.003418,0.003327,0.002967,0.004426,0.004529,0.004124
Energy,0.003240,0.003282,0.002668,0.003050,0.003121,0.003059,0.002832,0.002839,0.003853,0.004037,0.003490
Financials,0.004203,0.004403,0.003557,0.003121,0.004613,0.003898,0.003922,0.003706,0.005052,0.005914,0.004667
Health Care,0.003841,0.003930,0.003418,0.003059,0.003898,0.003920,0.003510,0.003281,0.004873,0.005053,0.004261
Industrials,0.003768,0.003802,0.003327,0.002832,0.003922,0.003510,0.003669,0.003244,0.004685,0.005045,0.004294
Information Technology,0.003587,0.003671,0.002967,0.002839,0.003706,0.003281,0.003244,0.003197,0.004266,0.004736,0.003885
Materials,0.005120,0.004975,0.004426,0.003853,0.005052,0.004873,0.004685,0.004266,0.006868,0.006716,0.005735
Real Estate,0.005444,0.005627,0.004529,0.004037,0.005914,0.005053,0.005045,0.004736,0.006716,0.007865,0.006037


Black-Litterman Model

In [ ]:
class BlackLittermanModel:
    def __init__(self, market_equilibrium_returns, covariance_matrix, views, confidence_matrix):
        self.market_equilibrium_returns = market_equilibrium_returns
        self.covariance_matrix = covariance_matrix
        self.views = views
        self.confidence_matrix = confidence_matrix

    def calculate_bl_expected_returns(self):
        """
        Calculate expected returns using the Black-Litterman model.
        """
        tau = 0.025  # Tau is the scaling factor, adjust as needed

        # Calculate the equilibrium excess returns
        pi = np.dot(self.covariance_matrix, self.market_equilibrium_returns)

        # Calculate the Black-Litterman expected returns
        bl_expected_returns = np.dot(
            np.linalg.inv(np.linalg.inv(tau * self.covariance_matrix) + np.dot(np.dot(self.views.T, np.linalg.inv(self.confidence_matrix)), self.views)),
            np.dot(np.linalg.inv(tau * self.covariance_matrix), pi) + np.dot(np.dot(self.views.T, np.linalg.inv(self.confidence_matrix)), self.views.T.dot(self.views.dot(self.market_equilibrium_returns)))
        )

        return bl_expected_returns

In [ ]:
# Assuming you have a risk-free rate and equity risk premium
risk_free_rate = 0.047  # Adjust based on the current risk-free rate
equity_risk_premium = 0.06  # Adjust based on economic indicators

# Calculate market equilibrium returns
market_equilibrium_returns = risk_free_rate + equity_risk_premium + sector_mean_returns.mean(axis=0)

# Print the market equilibrium returns
print("Market Equilibrium Returns:")
print(market_equilibrium_returns)


Market Equilibrium Returns:
Communication Services    0.127542
Consumer Discretionary    0.127338
Consumer Staples          0.123838
Energy                    0.119987
Financials                0.126094
Health Care               0.126538
Industrials               0.125425
Information Technology    0.126551
Materials                 0.128964
Real Estate               0.129646
Utilities                 0.127757
dtype: float64


In [ ]:
# Define the sectors in your portfolio
sectors = [
    'Communication Services',
    'Consumer Discretionary',
    'Consumer Staples',
    'Energy',
    'Financials',
    'Health Care',
    'Industrials',
    'Information Technology',
    'Materials',
    'Real Estate',
    'Utilities'
]

# Assuming you have len(sectors) assets in your portfolio
n_assets = len(sectors)

# Define your views based on your analysis
views = np.zeros(n_assets)

# Replace the values below with your own views for each sector
views[0] = 0.02  # Positive view on Communication Services
views[1] = -0.01  # Negative view on Consumer Discretionary
views[2] = 0.015   # Positive view on Consumer Staples
views[3] = -0.02  # Negative view on Energy
views[4] = 0.01  # Positive view on Financials
views[5] = 0.005  # Slightly positive view on Health Care
views[6] = -0.015  # Negative view on Industrials
views[7] = 0.03  # Positive view on Information Technology
views[8] = -0.01  # Negative view on Materials
views[9] = 0.005  # Slightly positive view on Real Estate
views[10] = -0.02  # Negative view on Utilities

print("Your Views:")
print(views)

# Ensure that the sum of views is zero for market equilibrium
views = views - np.mean(views)
print("Adjusted Views:")
print(views)

# Define confidence levels for each view
confidence_levels = {
    'Communication Services': 0.09,
    'Consumer Discretionary': 0.09,
    'Consumer Staples': 0.09,
    'Energy': 0.09,
    'Financials': 0.09,
    'Health Care': 0.09,
    'Industrials': 0.09,
    'Information Technology': 0.09,
    'Materials': 0.09,
    'Real Estate': 0.09,
    'Utilities': 0.09,
}

# Create a diagonal matrix using the confidence levels
confidence_matrix = np.diag([confidence_levels[sectors[i]] for i in range(n_assets)])


Your Views:
[ 0.02  -0.01   0.015 -0.02   0.01   0.005 -0.015  0.03  -0.01   0.005
 -0.02 ]
Adjusted Views:
[ 0.01909091 -0.01090909  0.01409091 -0.02090909  0.00909091  0.00409091
 -0.01590909  0.02909091 -0.01090909  0.00409091 -0.02090909]


In [ ]:
# Assuming you have market_equilibrium_returns, covariance_matrix, investor_views, and confidence_matrix defined
bl_model = BlackLittermanModel(market_equilibrium_returns, cov_matrix, views, confidence_matrix)
bl_expected_returns = bl_model.calculate_bl_expected_returns()

# Print the Black-Litterman expected returns
print("Black-Litterman Expected Returns:")
print(bl_expected_returns)

Black-Litterman Expected Returns:
[0.00588681 0.005852   0.00500902 0.00448697 0.0059594  0.0054491
 0.00533057 0.0049854  0.00716423 0.00772728 0.0064819 ]


LSTM Model

Combine Black-Litterman and LSTM

Calculate Allocations

 Evaluate Portfolio Performance